In [1]:
import pandas as pd
import json
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer

# Load conversation JSON
with open("HumanConversation.json", "r", encoding="utf-8") as f:
    chat_data = json.load(f)

df = pd.DataFrame(chat_data)  # columns: "prompt", "completion"
print(f"✅ Loaded chat dataset. Number of rows: {len(df)}")


c:\Users\prash\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



✅ Loaded chat dataset. Number of rows: 52


In [2]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

prompts = df["prompt"].tolist()
completions = df["completion"].tolist()

prompt_embeddings = embedder.encode(prompts, convert_to_numpy=True, show_progress_bar=True)
prompt_embeddings = prompt_embeddings / np.linalg.norm(prompt_embeddings, axis=1, keepdims=True)
print(f"✅ Generated embeddings. Shape: {prompt_embeddings.shape}")

Batches: 100%|██████████| 2/2 [00:00<00:00,  9.23it/s]

✅ Generated embeddings. Shape: (52, 384)


In [3]:
# Initialize ChromaDB
client = chromadb.CloudClient(
  api_key='ck-FLEMJN1q9G85JWFrL2tPqE4B7EqmrGFzuz5sSFcuCpxX',
  tenant='f9ac33d4-f162-4b85-b38d-1c8b719366e6',
  database='VECDB'
)

collection_name = "basic_chat"
if collection_name in [c.name for c in client.list_collections()]:
    collection = client.get_collection(name=collection_name)
else:
    collection = client.create_collection(name=collection_name)

ids = [f"chat_{i}" for i in range(len(prompts))]
metadatas = [{"type": "chat"} for _ in range(len(prompts))]

collection.add(
    documents=prompts,
    embeddings=prompt_embeddings.tolist(),
    ids=ids,
    metadatas=metadatas
)

print("✅ All chat prompts added to ChromaDB!")


✅ All chat prompts added to ChromaDB!


In [4]:
from sentence_transformers import util

def chat_response(query, top_k=3):
    # Embed query
    q_emb = embedder.encode([query], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)
    
    # Retrieve top-k prompts from collection
    results = collection.query(
        query_embeddings=q_emb.tolist(),
        n_results=top_k
    )

    docs = results.get("documents", [[]])[0]
    if not docs:
        return "Sorry, I don’t have a response for that."

    # Pick the first match
    idx = prompts.index(docs[0])
    return completions[idx]


In [5]:
print("\nSentenara ready! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = chat_response(user_input)
    print("Bot:", response)



Sentenara ready! Type 'exit' to quit.
Bot: Hello! How can I help you today?
Bot: Hi there! How are you doing?
